In [191]:
import math

In [222]:
"""atomic unit of micrograd. Basis for building neurons, layers and MLPs."""

class Value:
    
    def __init__(self, data: float, _children:()=None, _op:str=None):
        self.data = float(data)
        self._prev = set(_children) if _children else None
        self.grad = None
        self._op = _op
    
    def __add__(self, right):                
        right = right if isinstance(right, Value) else Value(right)        
        out = Value(self.data + right.data, _children=[self, right], _op='+')            
        return out
    
    def __sub__(self, right):
        right = right if isinstance(right, Value) else Value(right)        
        out = Value(self.data - right.data, _children=[self, right], _op='-')
        return out            
    
    def __mul__(self, right):
        right = right if isinstance(right, Value) else Value(right)
        out = Value(self.data * right.data, [self, right], '*')
        return out
    
    def __pow__(self, power):
        data = self.data**power
        return Value(data, None, f'**{power}')
    
    def __truediv__(self, right):        
        right = right if isinstance(right, Value) else Value(right)
        out = Value(self.data * right.data**-1, [self, right], '/')
        return out
    
    def tanh(self,):
        out = Value(math.tanh(self.data), [self], 'tanh')
        return out
    
    def relu(self):
        out = Value(0 if self.data < 0 else self.data, [self], 'relu')
        return out
    
    def __radd__(self, right):
        return self + right

    def __rsub__(self, right):
        return -1*self + right
    
    def __rmul__(self, right):
        return self * right
    
    def __rtruediv__(self, right):
        return self**-1 * right
        
    def __repr__(self):
        msg = f'Value(data = {str(self.data)}, grad = {str(self.grad)})'
        return msg

In [224]:
a = Value(3)
b = 9

In [225]:
c = a**b

In [227]:
c._op

'**9'